In [ ]:
# Check core SDK version number for debugging purposes
import azureml.core
from azureml.core.model import Model
from azureml.core.image import Image
from azureml.core.webservice import Webservice
print("SDK Version:", azureml.core.VERSION)

In [ ]:
subscription_id = ""
resource_group = ""
workspace_name = ""
workspace_region = '' # e.g. westeurope

In [ ]:
# import the Workspace class and check the azureml SDK version# impor 
from azureml.core.workspace import Workspace, WorkspaceException

#ws = Workspace.create(name = workspace_name,
#                      subscription_id = subscription_id,
#                      resource_group = resource_group, 
#                      location = workspace_region)
ws = Workspace.from_config()
ws.get_details()

In [ ]:
model = Model.register(model_path = "model.pkl",
                          model_name = "model.pkl",
                          #tags = ["pdm"],
                          description = "Predictive Maintenance multi-class classifier",
                          workspace = ws)

In [ ]:
%%writefile score.py

import json
from sklearn.externals import joblib
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('model.pkl')
    model = joblib.load(model_path)

def run(rawdata):
    try:
        data = json.loads(rawdata)
        df = pd.DataFrame(data=data)
        result = model.predict(df).tolist()
    except Exception as e:
        result = str(e)
    return result

In [ ]:
# this forces IPython to load the latest version
# of the score module

import importlib
# https://docs.python.org/3/library/importlib.html#importlib.invalidate_caches
importlib.invalidate_caches()
import score
importlib.reload(score)

In [ ]:
score.init()

In [ ]:
with open('sample.json', 'r') as sample_file:
  sample_json = sample_file.read()

score.run(sample_json)

In [ ]:
%%writefile env.yml

name: pdm
channels:
  - defaults
dependencies:
  - pip:
    - azureml-defaults
    - scikit-learn
    - numpy
    - inference-schema[numpy-support]
    - azureml-core
    - azureml-sdk


In [ ]:
from azureml.core import Environment

env = Environment.from_conda_specification(name='pdm-multi-class-image', file_path='env.yml')

# This is optional at this point
env.register(workspace=ws)

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py", environment=env)

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import WebserviceException

deployment_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4)
aci_service_name = 'pdm'

try:
    # if you want to get existing service below is the command
    # since aci name needs to be unique in subscription deleting existing aci if any
    # we use aci_service_name to create azure aci
    service = Webservice(ws, name=aci_service_name)
    if service:
        service.delete()
except WebserviceException as e:
    print()

service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

In [ ]:
print(service.state)

In [ ]:
predictions = service.run(input_data = sample_json)
print(predictions)

In [ ]:
vars(service)

## Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice

# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'pdm-aks' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [ ]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)